In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import yfinance as yf
import os
from google.colab import files

# Manually Upload CSV Instead of Using Kaggle
print("Upload your CSV file containing company data:")
uploaded = files.upload()

# Get the uploaded file name
csv_filename = list(uploaded.keys())[0]
print(f"Uploaded file: {csv_filename}")

def load_company_data(filepath):
    df = pd.read_csv(filepath)

    if 'Symbol' not in df.columns or 'Industry' not in df.columns:
        raise ValueError("CSV file must contain 'Symbol' and 'Industry' columns.")

    label_encoder = LabelEncoder()
    df['Industry Encoded'] = label_encoder.fit_transform(df['Industry'])
    return df['Symbol'].tolist(), df['Industry'].tolist(), df['Industry Encoded'].tolist()

def fetch_stock_data(ticker, start_date, end_date):
    try:
        data = yf.download(f"{ticker}.NS", start=start_date, end=end_date)
        return data if not data.empty else None
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def compute_features(stock_data):
    numeric_cols = ['Open', 'High', 'Low', 'Close']
    stock_data[numeric_cols] = stock_data[numeric_cols].astype(float)

    for window in [5, 10, 15, 20, 25, 30]:
        stock_data[f'MA_{window}'] = stock_data['Close'].rolling(window=window).mean()

    scaler = StandardScaler()
    stock_data['Normalised Close'] = scaler.fit_transform(stock_data[['Close']])

    stock_data['Return Ratio'] = (stock_data['Close'] - stock_data['Close'].shift(1)) / stock_data['Close'].shift(1)
    stock_data['Percentage Change Open'] = stock_data['Open'] / stock_data['Close'] - 1
    stock_data['Percentage Change High'] = stock_data['High'] / stock_data['Close'] - 1
    stock_data['Percentage Change Low'] = stock_data['Low'] / stock_data['Close'] - 1

    return stock_data

def align_with_reference(stock_data, reference_dates):
    start_ref, end_ref = reference_dates
    stock_dates = stock_data.index

    if stock_dates.min() > start_ref or stock_dates.max() < end_ref:
        return False
    return True

def main():
    start_date, end_date = "2022-01-10", "2025-01-10"
    companies, sector, sector_encoded = load_company_data(csv_filename)
    os.makedirs('Data_is_here', exist_ok=True)

    reference_data = fetch_stock_data("ONGC", start_date, end_date)
    if reference_data is None:
        print("Failed to fetch reference data.")
        return
    reference_dates = (reference_data.index.min(), reference_data.index.max())

    missing = []
    for ticker in companies:
        stock_data = fetch_stock_data(ticker, start_date, end_date)
        if stock_data is not None:
            if not align_with_reference(stock_data, reference_dates):
                print(f"Skipping {ticker} due to insufficient data.")
                missing.append(ticker)
                continue

            stock_data = compute_features(stock_data)

            idx = companies.index(ticker) if ticker in companies else -1
            if idx != -1:
                stock_data['Sector'] = sector[idx]
                stock_data['Sector Encoded'] = sector_encoded[idx]

            stock_data.to_csv(f'Data_is_here/{ticker}_data.csv')
            print(f"Processed and saved data for {ticker}.")
        else:
            print(f"Skipping {ticker} due to missing data.")
            missing.append(ticker)

    with open("Data_is_here/skipped_stocks.txt", "w") as file:
        for line in missing:
            file.write(line + "\n")

    print("All data processing completed.")

if __name__ == "__main__":
    main()


Upload your CSV file containing company data:


Saving ind_nifty500list.csv to ind_nifty500list.csv
Uploaded file: ind_nifty500list.csv
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Processed and saved data for 360ONE.


[*********************100%***********************]  1 of 1 completed


...

Processed and saved data for ZOMATO.


[*********************100%***********************]  1 of 1 completed


Processed and saved data for ZYDUSLIFE.


[*********************100%***********************]  1 of 1 completed

Processed and saved data for ECLERX.
All data processing completed.


In [ ]:
import os
import pandas as pd

file_path = "/content/Data_is_here"

# Ensure the directory exists
if not os.path.exists(file_path):
    print("Error: Directory does not exist.")
    exit()

# Get list of CSV files
files = [f for f in os.listdir(file_path) if f.endswith(".csv")]

for file in files:
    file_full_path = os.path.join(file_path, file)

    df = pd.read_csv(file_full_path)

    # Ensure 'Return Ratio' exists in DataFrame
    if 'Return Ratio' not in df.columns:
        print(f"Skipping {file}: 'Return Ratio' column not found.")
        continue

    # Create the Stock_Movement_Label column
    df['Stock_Movement_Label'] = (df['Return Ratio'].shift(-5) > 0).astype(int)

    # Save the modified file
    df.to_csv(file_full_path, index=False)
    print(f"Processed {file}")


Processed ASTRAL_data.csv
Processed DRREDDY_data.csv
Processed PIDILITIND_data.csv
Processed GVT&D_data.csv
Processed FINPIPE_data.csv
Processed CERA_data.csv
Processed TANLA_data.csv
Processed HEG_data.csv
Processed KNRCON_data.csv
Processed FSL_data.csv
Processed OIL_data.csv
Processed INDIANB_data.csv
Processed BANKBARODA_data.csv
Processed ALOKINDS_data.csv
Processed COCHINSHIP_data.csv
Processed BANKINDIA_data.csv
Processed GAEL_data.csv
Processed LTIM_data.csv
Processed AMBUJACEM_data.csv
Processed NTPC_data.csv
Processed TRIDENT_data.csv
Processed AUROPHARMA_data.csv
Processed MFSL_data.csv
Processed TVSMOTOR_data.csv
Processed J&KBANK_data.csv
Processed SJVN_data.csv
Processed VARROC_data.csv
Processed PAGEIND_data.csv
Processed PRAJIND_data.csv
Processed APOLLOHOSP_data.csv
Processed TATACHEM_data.csv
Processed RAILTEL_data.csv
Processed GSPL_data.csv
Processed ALKEM_data.csv
Processed FINCABLES_data.csv
Processed VBL_data.csv
Processed NYKAA_data.csv
Processed KARURVYSYA_data

In [ ]:
import os
import pandas as pd

# Define directories
file_path = "/content/Data_is_here"
save_path = "/content/Preprocessed_data"

# Ensure save directory exists
os.makedirs(save_path, exist_ok=True)

def set_data(file):
    df = pd.read_csv(file_path + '/' + file)

    # Fill NaN values with 0
    df.fillna(0, inplace=True)

    # Drop the first two rows (if they contain metadata)
    df = df.iloc[2:].reset_index(drop=True)

    # Remove unnamed columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Debugging: Check number of columns before renaming
    print(f"Processing file: {file}")
    print(f"Original columns: {df.columns.tolist()} ({len(df.columns)} columns)")

    # Expected column names
    expected_columns = [
        "Date",
        "Close", "High", "Low", "Open",
        "Volume",
        "MA_5", "MA_15", "MA_10", "MA_20", "MA_25", "MA_30",
        "Normalized_Close",
        "Return_Ratio", "Percentage_Change_Open", "Percentage_Change_High", "Percentage_Change_Low",
        "Sector",
        "Sector_Encoded",
        "Stock_Movement_Label"
    ]

    # Fix: Only rename columns if the count matches
    if len(df.columns) == len(expected_columns):
        df.columns = expected_columns
    else:
        print(f"❌ Column mismatch in {file}. Expected {len(expected_columns)} but found {len(df.columns)}. Skipping renaming.")
        return  # Skip further processing for this file

    # Drop unnecessary columns
    df = df.drop(columns=[ "Sector", "Volume", "Normalized_Close"], errors='ignore')

    # Convert to numeric
    for col in df.columns:
        if col != 'Date':  # Skip converting 'Date' to numeric
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Drop rows with NaN values and reset index
    df = df.dropna().reset_index(drop=True)

    # Save preprocessed file
    df.to_csv(save_path + '/' + file, index=False)
    print(f"✅ Processed {file} successfully.")

# Process all files in the directory
for file in os.listdir(file_path):
    set_data(file)


Processing file: ASTRAL_data.csv
Original columns: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume', 'MA_5', 'MA_10', 'MA_15', 'MA_20', 'MA_25', 'MA_30', 'Normalised Close', 'Return Ratio', 'Percentage Change Open', 'Percentage Change High', 'Percentage Change Low', 'Sector', 'Sector Encoded', 'Stock_Movement_Label'] (20 columns)
✅ Processed ASTRAL_data.csv successfully.
Processing file: DRREDDY_data.csv
Original columns: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume', 'MA_5', 'MA_10', 'MA_15', 'MA_20', 'MA_25', 'MA_30', 'Normalised Close', 'Return Ratio', 'Percentage Change Open', 'Percentage Change High', 'Percentage Change Low', 'Sector', 'Sector Encoded', 'Stock_Movement_Label'] (20 columns)
✅ Processed DRREDDY_data.csv successfully.
Processing file: PIDILITIND_data.csv
Original columns: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume', 'MA_5', 'MA_10', 'MA_15', 'MA_20', 'MA_25', 'MA_30', 'Normalised Close', 'Return Ratio', 'Percentage Change Open', 'Percentage Change High'

In [ ]:
import os

file_path = "/content/Data_is_here"

if os.path.exists(file_path):
    files = [file for file in os.listdir(file_path) if file.endswith(".csv")]
    print(f"Number of companies (CSV files): {len(files)}")
else:
    print(f"Directory {file_path} does not exist.")


df = pd.read_csv("/content/Data_is_here/POWERGRID_data.csv")  # Replace with an actual filename
print(df.head())  # Check if "Stock_Movement_Label" is updated

Number of companies (CSV files): 451
        Price               Close                High                 Low  \
0      Ticker        POWERGRID.NS        POWERGRID.NS        POWERGRID.NS   
1        Date                 NaN                 NaN                 NaN   
2  2022-01-10  128.63702392578125     130.68638827754  128.29020328847753   
3  2022-01-11  128.54246520996094  129.77209180444058   128.3217669480974   
4  2022-01-12  129.23606872558594   130.3395726497143  128.85772708883846   

                 Open        Volume  MA_5  MA_10  MA_15  MA_20  MA_25  MA_30  \
0        POWERGRID.NS  POWERGRID.NS   NaN    NaN    NaN    NaN    NaN    NaN   
1                 NaN           NaN   NaN    NaN    NaN    NaN    NaN    NaN   
2     130.68638827754      11085710   NaN    NaN    NaN    NaN    NaN    NaN   
3  128.98387456274466       7208789   NaN    NaN    NaN    NaN    NaN    NaN   
4  129.89817621168464      11018139   NaN    NaN    NaN    NaN    NaN    NaN   

   Normalised Close

In [ ]:
pip install feedparser

In [ ]:
import pandas as pd

# Get NIFTY 500 stock list from NSE
nifty500_url = "https://archives.nseindia.com/content/indices/ind_nifty500list.csv"
df = pd.read_csv(nifty500_url)

# Extract company names and symbols
nifty500_stocks = df["Company Name"].tolist()

nifty500_symbols = df["Symbol"].tolist()

print(nifty500_stocks[:5])  # Check first 5 companies

In [ ]:
import feedparser
import pandas as pd
import concurrent.futures
import time
import random
from datetime import datetime, timedelta
from urllib.parse import quote  # ✅ URL encoding
import os

# Load NIFTY 500 company names
df = pd.read_csv(nifty500_url)
companies = df["Company Name"].tolist()

# CSV file to store results
csv_filename = "nifty500_google_rss_news.csv"

# Create CSV with headers if it doesn't exist
if not os.path.exists(csv_filename):
    pd.DataFrame(columns=["Date", "Company", "Headline", "Link"]).to_csv(csv_filename, index=False)

def get_google_rss_news(company_name, start_year=2022):
    """Fetch Google RSS news for a company, getting 1 headline per month from 2022 onward."""
    news_items = []
    query = quote(f"{company_name} stock")
    start_date = datetime(start_year, 1, 1)
    end_date = datetime.today()

    while start_date <= end_date:
        after_date = start_date.strftime("%Y-%m-%d")
        before_date = (start_date + timedelta(days=30)).strftime("%Y-%m-%d")
        rss_url = (f"https://news.google.com/rss/search?q={query}+after:{after_date}+before:{before_date}"
                   f"&hl=en-IN&gl=IN&ceid=IN:en")

        feed = feedparser.parse(rss_url)

        if feed.entries:
            entry = feed.entries[0]
            news_items.append({
                "Date": datetime(*entry.published_parsed[:3]).strftime("%Y-%m-%d"),
                "Company": company_name,
                "Headline": entry.title,
                "Link": entry.link
            })
#        print(f"📰 {news_item['Date']} | {news_item['Company']} | {news_item['Headline']}")
        start_date += timedelta(days=30)

        # 🔹 Avoid Google rate limits
        time.sleep(random.uniform(1, 3))

    return news_items

# Run scraper and save after each company
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:  # Adjust `max_workers`
    future_to_company = {executor.submit(get_google_rss_news, company): company for company in companies}

    for future in concurrent.futures.as_completed(future_to_company):
        company_name = future_to_company[future]
        try:
            company_news = future.result()
            if company_news:
                # Convert to DataFrame and **append to CSV**
                pd.DataFrame(company_news).to_csv(csv_filename, mode="a", header=False, index=False)
                print(f"✅ Saved news for {company_name}")
            else:
                print(f"⚠️ No news for {company_name}")
        except Exception as e:
            print(f"❌ Error fetching news for {company_name}: {e}")

print("✅ Scraping complete! News saved incrementally to nifty500_google_rss_news.csv")


In [ ]:
!pip install vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import feedparser
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import quote  # ✅ URL encoding
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to get normalized sentiment score (0 to 1)
def get_sentiment_score(text):
    compound_score = analyzer.polarity_scores(text)['compound']
    return (compound_score + 1) / 2  # Normalize to 0-1 range


# Apply sentiment analysis to each headline
news_df=pd.read_csv("nifty500_google_rss_news.csv")
news_df["Sentiment Score"] = news_df["Headline"].apply(get_sentiment_score)

# Save results with sentiment scores
news_df.to_csv("nifty500_weekly_google_rss_news_with_sentiment.csv", index=False)

print("✅ Sentiment analysis complete! CSV saved with sentiment scores in the range [0,1].")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Example path to your CSV file on Google Drive
csv_path = '/content/drive/MyDrive/nifty500_weekly_google_rss_news_with_sentiment.csv'
df = pd.read_csv(csv_path)


In [ ]:
import tensorflow as tf

# Create a tensor on the GPU
with tf.device('/device:GPU:0'):
    tensor_gpu = tf.constant([1, 2, 3])

# Move the tensor to the CPU
with tf.device('/device:CPU:0'):
    tensor_cpu = tf.identity(tensor_gpu)

print(tensor_cpu.device)


In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from scipy.special import softmax
import os
from google.colab import drive

# Mount Google Drive (force remount if already mounted)
drive.mount('/content/drive', force_remount=True)

# List all files in the MyDrive directory to verify the file exists
print("Files in MyDrive:")
print(os.listdir('/content/drive/MyDrive'))

# Example path to your CSV file on Google Drive
csv_path = '/content/drive/MyDrive/nifty500_google_rss_news.csv'

# Check if the file exists
if not os.path.exists(csv_path):
    print("File not found. Please ensure it is uploaded to Google Drive and the path is correct.")
else:
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Ensure 'Headline' column exists
    if 'Headline' not in df.columns:
        print("The CSV file must have a column named 'Headline'.")
        exit()

    # Load FinBERT model & tokenizer
    MODEL_NAME = "ProsusAI/finbert"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

    # Function to get sentiment score from FinBERT
    def get_finbert_sentiment(text):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        scores = softmax(outputs.logits.numpy()[0])  # Convert logits to probabilities
        sentiment_score = scores[2]  # Positive sentiment score
        return sentiment_score  # Returns a value between 0 and 1

    # Remove existing sentiment analysis column if it exists
    if 'Sentiment Score' in df.columns:
        df = df.drop('Sentiment Score', axis=1)

    # Adjust batch size based on available memory and dataset size
    batch_size = 300  # Process in batches of up to 500 or 10% of total rows

    # Apply sentiment analysis in batches
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        df.loc[i:i+batch_size, "Sentiment Score"] = batch["Headline"].apply(get_finbert_sentiment)
        print(f"✅ Processed batch {i} to {i+batch_size}")

    # Save results
    df.to_csv('/content/drive/MyDrive/sentiment_analysis_results.csv', index=False)
    print("✅ Sentiment analysis complete! Results saved.")


In [ ]:
# from google.colab import drive
# import pandas as pd

# # Mount Google Drive
# drive.mount('/content/drive')

# # Specify the file path
# file_path = "/content/drive/MyDrive/sentiment_scores.csv"

# # Load the CSV file
# df = pd.read_csv(file_path)
# print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         Date        Company  Sentiment Score
0  30/03/2022  3M India Ltd.          0.50000
1  12/04/2022  3M India Ltd.          0.81245
2  10/05/2022  3M India Ltd.          0.50000
3  22/06/2022  3M India Ltd.          0.50000
4  21/07/2022  3M India Ltd.          0.65910


In [ ]:
# # List all files in the preprocessed directory
# print("Files in preprocessed directory:")
# for file in files:
#     print(file)


Files in preprocessed directory:
ASTRAL_data.csv
DRREDDY_data.csv
PIDILITIND_data.csv
GVT&D_data.csv
FINPIPE_data.csv
CERA_data.csv
TANLA_data.csv
HEG_data.csv
KNRCON_data.csv
FSL_data.csv
OIL_data.csv
INDIANB_data.csv
BANKBARODA_data.csv
ALOKINDS_data.csv
COCHINSHIP_data.csv
BANKINDIA_data.csv
GAEL_data.csv
LTIM_data.csv
AMBUJACEM_data.csv
NTPC_data.csv
TRIDENT_data.csv
AUROPHARMA_data.csv
MFSL_data.csv
TVSMOTOR_data.csv
J&KBANK_data.csv
SJVN_data.csv
VARROC_data.csv
PAGEIND_data.csv
PRAJIND_data.csv
APOLLOHOSP_data.csv
TATACHEM_data.csv
RAILTEL_data.csv
GSPL_data.csv
ALKEM_data.csv
FINCABLES_data.csv
VBL_data.csv
NYKAA_data.csv
KARURVYSYA_data.csv
ASTRAZEN_data.csv
GODREJAGRO_data.csv
HCLTECH_data.csv
RHIM_data.csv
SONATSOFTW_data.csv
VOLTAS_data.csv
VINATIORGA_data.csv
SKFINDIA_data.csv
IIFL_data.csv
TATAMOTORS_data.csv
RATNAMANI_data.csv
JPPOWER_data.csv
HINDZINC_data.csv
BPCL_data.csv
CRAFTSMAN_data.csv
PHOENIXLTD_data.csv
SWANENERGY_data.csv
TATASTEEL_data.csv
BALRAMCHIN_data.csv

In [ ]:
# import pandas as pd

# sentiment_file = "/content/drive/MyDrive/sentiment_scores.csv"  # Correct path to sentiment scores

# # Load the sentiment scores data, reading 'Company' as a string
# sentiment_df = pd.read_csv(sentiment_file, dtype={'Company': str})

# # Clean company names in sentiment_df
# sentiment_df['Company'] = (
#     sentiment_df['Company']
#     .str.upper()
#     .str.replace(r'LTD\.?', '', regex=True)  # Removes "LTD." and "LTD"
#     .str.replace(r' ?LTD', '', regex=True)   # Removes " LTD"
#     .str.replace(r'\(.*?\)', '', regex=True)  # Removes parentheses and content inside
#     .str.replace(r'[^A-Z0-9&\s]+', '', regex=True) # Remove non-alphanumeric except spaces and &
#     .str.replace(' +', ' ', regex=True)    # Replace multiple spaces with a single space
#     .str.strip()
# )

# # Loop through each company name in the column and remove spaces
# for index, row in sentiment_df.iterrows():
#     sentiment_df.at[index, 'Company'] = row['Company'].replace(" ", "")

# # Print the unique company names in the 'Company' column
# print(sentiment_df['Company'].unique())


['3MINDIA' '360ONEWAM' 'ABBINDIA' 'AIAENGINEERING' 'ACC' 'APLAPOLLOTUBES'
 'AADHARHOUSINGFINANCE' 'AUSMALLFINANCEBANK' 'AARTIINDUSTRIES'
 'AAVASFINANCIERS' 'ADANIENERGYSOLUTIONS' 'ABBOTTINDIA'
 'ACTIONCONSTRUCTIONEQUIPMENT' 'ADANIENTERPRISES' 'ADANIGREENENERGY'
 'ADANIPORTSANDSPECIALECONOMICZONE' 'ADANIWILMAR' 'ADANIPOWER'
 'ADANITOTALGAS' 'ADITYABIRLACAPITAL' 'ADITYABIRLAFASHIONANDRETAIL'
 'ADITYABIRLAREALESTATE' 'AEGISLOGISTICS' 'ADITYABIRLASUNLIFEAMC' 'AFFLE'
 'AJANTAPHARMACEUTICALS' 'AKUMSDRUGSANDPHARMACEUTICALS'
 'ALKEMLABORATORIES' 'ALEMBICPHARMACEUTICALS' 'ALKYLAMINESCHEMICALS'
 'ALOKINDUSTRIES' 'ANANDRATHIWEALTH' 'AMBERENTERPRISESINDIA'
 'AMARARAJAENERGY&MOBILITY' 'AMBUJACEMENTS' 'ANANTRAJ' 'APARINDUSTRIES'
 'ANGELONE' 'APOLLOHOSPITALSENTERPRISE' 'APOLLOTYRES'
 'APTUSVALUEHOUSINGFINANCEINDIA' 'ARCHEANCHEMICALINDUSTRIES'
 'ASAHIINDIAGLASS' 'ASHOKLEYLAND' 'ASIANPAINTS' 'ASTERDMHEALTHCARE'
 'ASTRAZENCAPHARMAINDIA' 'ASTRAL' 'ATUL' 'AUROBINDOPHARMA' 'AVANTIFEEDS'
 'AVENUESUPERMARTS'

In [ ]:
# import pandas as pd
# import os
# import re
# from google.colab import drive  # Import Google Drive library

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define directories
# file_path = "/content/Preprocessed_data"  # **IMPORTANT: REPLACE WITH THE CORRECT PATH**
# sentiment_file = "/content/drive/MyDrive/sentiment_scores.csv"  # **REPLACE WITH THE CORRECT PATH TO YOUR FILE IN DRIVE**
# output_sentiment_file = "/content/drive/MyDrive/cleaned_sentiment_scores.csv"  # New file in Drive

# # 1. Load Data and Preprocess
# def load_and_preprocess_data(file_path, sentiment_file):
#     """Loads the stock and sentiment data, preprocesses them, and returns them."""
#     try:
#         stock_filenames = [f[:-4] for f in os.listdir(file_path) if f.endswith('.csv')]
#     except FileNotFoundError:
#         print(f"ERROR: Directory not found: {file_path}")
#         return None, None

#     try:
#         sentiment_data = pd.read_csv(sentiment_file)
#         sentiment_data['Date'] = pd.to_datetime(sentiment_data['Date'], format='%d/%m/%Y')
#     except FileNotFoundError:
#         print(f"ERROR: Sentiment file not found: {sentiment_file}")
#         return None, None

#     return stock_filenames, sentiment_data


# # 2. Clean Company Names in Sentiment Data and Save
# def clean_and_save_sentiment_data(sentiment_data, output_sentiment_file):
#     """Cleans the company names in the sentiment data and saves to a new CSV."""
#     if sentiment_data is None:
#         return None

#     # Clean the company names using the same logic as before
#     def clean_company_name(company_name):
#         cleaned_name = re.sub(r"(ltd\.?|limited|india)", "", company_name, flags=re.IGNORECASE).strip()
#         return cleaned_name

#     sentiment_data['Company'] = sentiment_data['Company'].apply(clean_company_name)

#     # Save the cleaned sentiment data to a new CSV file in Google Drive
#     try:
#         sentiment_data.to_csv(output_sentiment_file, index=False)
#         print(f"✅ Cleaned sentiment data saved to: {output_sentiment_file}")
#     except Exception as e:
#         print(f"❌ Error saving cleaned sentiment data: {e}")
#         return None

#     return sentiment_data

# # 3. Create Company Name Mapping
# def create_company_name_mapping(stock_filenames, sentiment_data):
#     """Creates a dictionary mapping cleaned sentiment company names to stock filenames."""
#     if stock_filenames is None or sentiment_data is None:
#         return {}

#     sentiment_company_names = sentiment_data['Company'].unique()
#     mapping = {}

#     for sentiment_name in sentiment_company_names:
#         # Clean each stock filename
#         cleaned_stock_filenames = [re.sub(r"_", " ", f) for f in stock_filenames]  # Replace underscores with spaces

#         # Try a direct match (case-insensitive)
#         direct_match_index = next((i for i, f in enumerate(cleaned_stock_filenames) if sentiment_name.lower() == f.lower()), None)

#         if direct_match_index is not None:
#             mapping[sentiment_name] = stock_filenames[direct_match_index]  # map using the original stock filename
#             continue

#         # If still no match, print and require manual mapping
#         print(f"⚠️  Manual mapping required for: '{sentiment_name}'")
#         mapping[sentiment_name] = None  # Placeholder - you'll need to fill this in manually

#     return mapping

# # 4. Apply the Mapping and Merge
# def apply_mapping_and_merge(stock_filenames, sentiment_data, mapping):
#     """Applies the company name mapping and merges the stock and sentiment data."""
#     if stock_filenames is None or sentiment_data is None:
#         print("❌ Cannot merge because loading failed.")
#         return None

#     # Verify mapping is complete (no None values)
#     if any(value is None for value in mapping.values()):
#         print("❌ Mapping is incomplete.  Please manually update the 'mapping' dictionary with correct values.")
#         return None

#     # Apply the mapping to the sentiment data
#     sentiment_data['Company'] = sentiment_data['Company'].map(mapping)
#     sentiment_data.dropna(subset=['Company'], inplace=True)  # Remove rows with no match

#     # Filter for the 3-year period
#     start_date = pd.to_datetime('2022-03-28')
#     end_date = pd.to_datetime('2025-03-28')
#     sentiment_data = sentiment_data[(sentiment_data['Date'] >= start_date) & (sentiment_data['Date'] <= end_date)]

#     # Create a temporary dataframe to merge
#     merged_data_list = []
#     for filename in stock_filenames:
#         try:
#             stock_data = pd.read_csv(os.path.join(file_path, filename + '.csv'))
#             stock_data['Date'] = pd.to_datetime(stock_data['Date'])
#             stock_data = stock_data[(stock_data['Date'] >= start_date) & (stock_data['Date'] <= end_date)]
#             stock_data = stock_data.set_index('Date').resample('M').mean().reset_index()
#             stock_data['Company'] = filename
#             merged_data = pd.merge(stock_data, sentiment_data[sentiment_data['Company'] == filename], on=['Date', 'Company'], how='left')
#             merged_data_list.append(merged_data)
#         except FileNotFoundError:
#             print(f"❌ Stock data file not found: {filename}.csv")
#         except Exception as e:
#             print(f"Error processing {filename}: {e}")

#     # Concatenate all the merged dataframes
#     final_merged_data = pd.concat(merged_data_list, ignore_index=True)

#     return final_merged_data


# # Main Execution
# # Load data
# stock_filenames, sentiment_data = load_and_preprocess_data(file_path, sentiment_file)

# # Clean and save sentiment data
# sentiment_data = clean_and_save_sentiment_data(sentiment_data, output_sentiment_file)

# # Create mapping
# mapping = create_company_name_mapping(stock_filenames, sentiment_data)

# # Manually fix for cases where auto mapping fails
# mapping = {
#     '3M India': '3M_India',
#     '360 ONE WAM': '360_ONE_WAM',
#     'ABB India': 'ABB_India',
#     'AIA Engineering': 'AIA_Engineering',
#     'ACC': 'ACC',
#     'APL Apollo Tubes': 'APL_Apollo_Tubes',
#     'Aadhar Housing Finance': 'Aadhar_Housing_Finance',
#     'AU Small Finance Bank': 'AU_Small_Finance_Bank',
#     'Aarti Industries': 'Aarti_Industries',
#     'Aavas Financiers': 'Aavas_Financiers',
#     'Adani Energy Solutions': 'Adani_Energy_Solutions',
#     'Abbott India': 'Abbott_India'
# }

# # Apply mapping and merge
# final_merged_data = apply_mapping_and_merge(stock_filenames, sentiment_data, mapping)

# if final_merged_data is not None:
#     print("✅ Merged data successfully.")
#     print(final_merged_data.head())  # print the head to show how merged_data looks
#     # Save the merged data
#     final_merged_data.to_csv("merged_data.csv", index=False)
# else:
#     print("❌ Merging failed. Check the mapping and errors.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Cleaned sentiment data saved to: /content/drive/MyDrive/cleaned_sentiment_scores.csv
⚠️  Manual mapping required for: '3M'
⚠️  Manual mapping required for: '360 ONE WAM'
⚠️  Manual mapping required for: 'ABB'
⚠️  Manual mapping required for: 'AIA Engineering'
⚠️  Manual mapping required for: 'ACC'
⚠️  Manual mapping required for: 'APL Apollo Tubes'
⚠️  Manual mapping required for: 'Aadhar Housing Finance'
⚠️  Manual mapping required for: 'AU Small Finance Bank'
⚠️  Manual mapping required for: 'Aarti Industries'
⚠️  Manual mapping required for: 'Aavas Financiers'
⚠️  Manual mapping required for: 'Adani Energy Solutions'
⚠️  Manual mapping required for: 'Abbott'
⚠️  Manual mapping required for: 'Action Construction Equipment'
⚠️  Manual mapping required for: 'Adani Enterprises'
⚠️  Manual mapping required for: 'Adani Green Energy'
⚠️  Manual mapping required

<ipython-input-26-923bd599cdd5>:110: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  stock_data = stock_data.set_index('Date').resample('M').mean().reset_index()
<ipython-input-26-923bd599cdd5>:110: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  stock_data = stock_data.set_index('Date').resample('M').mean().reset_index()
<ipython-input-26-923bd599cdd5>:110: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  stock_data = stock_data.set_index('Date').resample('M').mean().reset_index()
<ipython-input-26-923bd599cdd5>:110: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  stock_data = stock_data.set_index('Date').resample('M').mean().reset_index()
<ipython-input-26-923bd599cdd5>:110: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  st

✅ Merged data successfully.
        Date        Close         High          Low         Open         MA_5  \
0 2022-03-31  1499.401917  1520.192080  1471.628760  1492.307122  1492.812445   
1 2022-04-30  1568.859356  1591.668214  1543.045563  1567.744524  1557.861809   
2 2022-05-31  1330.835234  1362.927507  1312.393526  1349.784803  1362.685293   
3 2022-06-30  1249.152499  1271.510651  1227.704605  1252.137540  1252.329084   
4 2022-07-31  1302.784656  1317.099395  1283.842192  1302.195093  1292.938191   

         MA_15        MA_10        MA_20        MA_25        MA_30  \
0  1498.399704  1477.476166  1456.575021  1443.486898  1438.511169   
1  1543.178449  1531.529471  1518.234054  1504.980364  1491.433172   
2  1404.946351  1442.602399  1472.145252  1488.982143  1495.947538   
3  1253.331896  1257.889070  1267.110717  1282.803353  1306.057056   
4  1283.892973  1273.305058  1265.126312  1261.744409  1260.451285   

   Return_Ratio  Percentage_Change_Open  Percentage_Change_High 